# Laboratorio: Applicazioni Web con Flask

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Setup

- Eseguire l'esercitazione **localmente** sul PC di laboratorio o sul proprio PC, non utilizzare Colab o simili
- Creare una nuova directory per l'esercitazione e salvare ed eseguire in essa questo file
- Aprire un terminale ed impostare come directory di lavoro (`cd`) quella creata sopra
- Se si vuole utilizzare un ambiente virtuale, attivarlo
- Accertarsi che, oltre a scikit-learn e le altre librerie di uso comune, siano installati Flask e xlrd (per la lettura dei file Excel)

In [1]:
pip install flask xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 672.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01
You should consider upgrading via the '/home/rrobby/software/jupyter/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Uso di modelli predittivi in applicazioni Flask

- **Flask** è un microframework per la realizzazione di applicazioni Web
- Per realizzare un'applicazione con funzionalità minime sono sufficienti poche righe di codice
- Nelle applicazioni Flask si possono integrare tutte le funzionalità di Python e librerie esterne, incluso l'utilizzo di modelli di classificazione e regressione
- In questa esercitazione vediamo come usare Flask per creare una semplice interfaccia per consultare un modello di classificazione

## Caso di studio: Predizione default carte di credito

- La predizione del rischio di credito è importante per le banche per stimare in anticipo il capitale che avranno a disposizione
- Dati dei titolari di carte di credito, vogliamo predire per ogni mese se rimborseranno puntualmente le spese effettuate
- Vogliamo addestrare un modello a predire i mancati rimborsi (_default_) basandosi sulle informazioni disponibili
- Una volta addestrato un modello, vogliamo creare una semplice interfaccia Web dove un operatore possa
  - inserire i valori noti delle variabili predittive
  - visualizzare la predizione del modello

### Reperimento dati

- Su https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients è fornito un dataset che riassume lo stato di 30.000 titolari di carte di credito di una banca in Taiwan nel 2005
- Carichiamo i dati direttamente dal file Excel dove sono forniti
  - pandas fornisce la funzione `read_excel`, per utilizzarla deve essere installata la libreria `xlrd`

In [2]:
import os.path
if not os.path.exists("ccdefault.xls"):
    from urllib.request import urlretrieve
    urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls", "ccdefault.xls")

In [3]:
import pandas as pd
ccd = pd.read_excel("ccdefault.xls", index_col=0, skiprows=1)

In [4]:
ccd.head(3)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0


### Significato delle colonne

Riportiamo le informazioni sul dataset tratte dalla [fonte](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)

- **`ID`** è un identificativo unico di ciascun cliente

I seguenti attributi sono di tipo categorico (nominale o ordinale), ma codificati come numeri:
- **`SEX`**: sesso
  - 1 = maschio, 2 = femmina
- **`EDUCATION`**: titolo di studio
  - 1 = laurea specialistica, 2 = laurea breve, 3 = scuola superiore, 4 = altro
- **`MARRIAGE`**: stato civile
  - 1 = sposato, 2 = nubile/celibe, 3 = altro
- **`default payment next month`**: stato pagamento dell'ultimo mese **(variabile da prevedere!)**
  - 0 = pagato, 1 = insoluto

- Per semplificare la procedura e l'interfaccia della webapp, in questo esempio ci limitiamo ad usare tre variabili predittive senza preprocessing:
  - `LIMIT_BAL`: cifra da pagare
  - `PAY_6`: ritardo nell'ultimo pagamento
  - `EDUCATION`: livello di educazione
- Estraiamo due strutture dati X e y con le tre variabili predittive e la variabile da predire

In [5]:
X = ccd[["LIMIT_BAL", "PAY_6", "EDUCATION"]]
y = ccd["default payment next month"]

## Creare un'applicazione Flask da zero

- Vediamo passo per passo come creare una webapp Flask
- Per iniziare, creiamo il modulo o package che conterrà l'app
- Essendo un'applicazione semplice, utilizziamo un modulo, ovvero un singolo file `.py`
- All'interno del file creiamo un oggetto di tipo `Flask` che rappresenta l'app
- Eseguire la cella di codice seguente per creare un file `predict.py` col contenuto indicato sotto

In [6]:
%%writefile predict.py
from flask import Flask, request, render_template

app = Flask(__name__)

Writing predict.py


## Esempio di richiesta e risposta

- In una webapp dobbiamo definire delle funzioni che rispondano alle richieste in arrivo dai client
- Per far sì che una funzione sia eseguita a seguito di determinate richieste, la decoriamo con `@app.route(percorso_pagina)`
- Ad esempio creiamo una funzione che restituisca un semplice messaggio "Hello!"
- Associamo tale funzione al percorso `/` della webapp, ovvero la home page
- Eseguire la cella seguente per aggiungerne il contenuto in fondo (_append_, `-a`) al file `predict.py` creato sopra

In [7]:
%%writefile -a predict.py

@app.route("/")
def home():
    return "Hello!"

Appending to predict.py


## Eseguire l'applicazione

Per eseguire l'applicazione nel terminale dobbiamo:
- impostare una variabile d'ambiente `FLASK_APP` col nome del modulo dell'applicazione
  - Windows: `set FLASK_APP=predict`
  - Mac OS / Linux: `export FLASK_APP=predict`
- abilitare la modalità debug impostando `FLASK_ENV`
  - Windows: `set FLASK_ENV=development`
  - Mac OS / Linux: `export FLASK_ENV=development`
- avviare il webserver di Flask
  - `flask run` (se non funziona, provare `python -m flask run`)

## Testare l'applicazione

- Sul browser, aprire l'URL http://127.0.0.1:5000/
- Dovrebbe comparire la scritta "Hello!" restituita dalla funzione `home()`
- Avendo abilitato la modalità debug, qualsiasi cambiamento nel codice si dovrebbe riflettere immediatamente sulla webapp
  - provare a cambiare la stringa "Hello!" nel codice, salvare il file e ricaricare la pagina

## Richiesta con parametri

- Per consultare un modello di predizione, accetteremo una richiesta dove gli input al modello sono passati come parametri
- In una richiesta GET è possibile passare i parametri accodandoli all'URL nella forma `?p1=val1&p2=val2&...`
- Possiamo accedere a questi parametri da Flask tramite il dizionario `request.args`
- Definiamo una funzione `predict` richiamata dal percorso `/predict` che prenda come parametri i valori delle tre variabili predittive considerate sopra e restituisca la predizione
  - i valori dei parametri sono reperiti come stringhe, vanno convertiti in valori numerici
  - per semplicità diamo a ciascun parametro lo stesso nome della variabile, ma non è necessario (è importante invece l'ordine in cui sono letti)
  - per ora come predizione restituiamo arbitrariamente "Non insolvente"
- Eseguire la seguente cella per aggiungere la funzione

In [8]:
%%writefile -a predict.py

@app.route("/predict")
def predict():
    inputs = [
        float(request.args["LIMIT_BAL"]),
        int(request.args["PAY_6"]),
        int(request.args["EDUCATION"])
    ]
    return "Non insolvente"

Appending to predict.py


- Possiamo testare la funzione facendo una richiesta a `predict` con valori arbitrari dei parametri, ad es.:
  - http://127.0.0.1:5000/predict?LIMIT_BAL=20000&PAY_6=-1&EDUCATION=1

## Utilizzare un modello nella webapp

- Abbiamo visto finora come addestrare modelli di predizione e come valutarli e/o utilizzarli all'interno della stessa sessione di lavoro
- In un'applicazione Web ogni richiesta è gestita da un processo separato, per cui non abbiamo accesso ad alcun modello addestrato in precedenza
  - non possiamo ripetere l'addestramento di un modello ad ogni richiesta, in quanto i tempi di risposta sarebbero lunghi
- La soluzione è **salvare il modello in un file** una volta addestrato, in modo che possa essere ricaricato ad ogni richiesta in tempi trascurabili

## Serializzazione di oggetti Python

- Il modulo `pickle` della libreria standard di Python fornisce funzionalità per la (de)serializzazione di oggetti Python
- Una volta addestrato un modello possiamo **salvarne in un file una rappresentazione binaria**
- Quando necessario possiamo ricaricare tale rappresentazione dal file e ricostruire da essa il modello addestrato

### Addestramento e salvataggio di un modello

- Definiamo un modello di classificazione, ad es. un perceptron con standardizzazione dei dati
  - si assume questo sia il modello migliore scelto in seguito ad una cross validation

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
model = Pipeline([
    ("scaler", StandardScaler()),
    ("model",  Perceptron())
])

- Addestriamo tale modello sul dataset
  - possiamo usare tutti i dati assumendo di aver già eseguito la validazione del modello

In [10]:
model.fit(X, y);

- Vediamo i coefficienti del modello, che indicano l'importanza di ciascuna variabile

In [11]:
model.named_steps["model"].coef_

array([[-0.11961879,  3.47835627,  2.53057092]])

- Utilizziamo ora il modulo `pickle` per serializzare il modello
- Apriamo un file `model.bin` per la scrittura (w) in formato binario (b)
- Usiamo la funzione `dump` di `pickle` per scrivere la rappresentazione binaria del modello addestrato all'interno del file

In [12]:
import pickle
with open("model.bin", "wb") as f:
    pickle.dump(model, f)

- Abbiamo così salvato il modello all'interno del file

### Caricamento di un modello

- Vediamo ora come caricare il modello dal file
- Apriamo di nuovo il file binario `model.bin`, questa volta in lettura (r)
- Per leggere il contenuto del file e riottenere il modello come oggetto Python usiamo la funzione `load` di `pickle`
- Salviamo l'oggetto in una nuova variabile `loaded_model`

In [13]:
with open("model.bin", "rb") as f:
    loaded_model = pickle.load(f)

- Possiamo verificare che `loaded_model` sia equivalente a `model`, ad esempio verificando che i coefficienti coincidano

In [15]:
loaded_model.named_steps["model"].coef_

array([[-0.11961879,  3.47835627,  2.53057092]])

## Caricamento del modello nella webapp

- Nella funzione `predict` della webapp possiamo quindi caricare tale modello ed utilizzarlo per ottenere una predizione vera e propria
- Per caricare un file dalla directory della webapp Flask (indipendentemente da quella di lavoro), usiamo il metodo `app.open_resource` al posto di `open`
- Una volta ricaricato il modello, otteniamo una singola predizione per i dati inseriti e visualizziamo come risposta una stringa "Non insolvente" (0) o "Insolvente" (1)

- Modificare la funzione `predict` in questo modo:

```
@app.route("/predict")
def predict():
    inputs = [
        float(request.args["LIMIT_BAL"]),
        int(request.args["PAY_6"]),
        int(request.args["EDUCATION"])
    ]
    with app.open_resource("model.bin", "rb") as f:
        model = pickle.load(f)
    output = model.predict([inputs])[0]
    return "Insolvente" if output else "Non insolvente"
```

- Verificare il funzionamento, usando ad esempio le seguenti richieste:
  - http://127.0.0.1:5000/predict?LIMIT_BAL=20000&PAY_6=-1&EDUCATION=1
  - http://127.0.0.1:5000/predict?LIMIT_BAL=500000&PAY_6=7&EDUCATION=1

## Dare un'interfaccia alla webapp

- Per ora la webapp restituisce semplici stringhe e interagiamo con essa scrivendo degli URL
- Per ottenere una webapp usabile, dobbiamo restituire pagine HTML al posto di semplici stringhe
- Tali pagine possono contenere link per navigare alle altre pagine senza scrivere manualmente gli URL
- Per definire le pagine creiamo dei _template_ HTML, all'interno dei quali possiamo integrare dati calcolati via codice
- Per iniziare, creiamo la directory `templates` dove salvare i template

In [15]:
import os
if not os.path.isdir("templates"):
    os.mkdir("templates")

- Creiamo ora un template minimale `predict.html` dove viene visualizzata la predizione
  - il file è una pagina HTML minimale con un segnaposto `{{ resp }}` al posto del quale visualizzare "Non insolvente" o "Insolvente"

In [16]:
%%writefile templates/predict.html
<!DOCTYPE html>
<head>
 <title>Predizione default carte</title>
</head>
<body>
 <h2>Default carte di credito</h2>
 <p>Predizione: <b>{{ resp }}</b></p>
</body>
</html>

Overwriting templates/predict.html


- Modifichiamo ora la funzione `predict` in modo che visualizzi la risposta nel template
- Usiamo la funzione `render_template` per ottenere la pagina HTML, passando come argomento il valore da dare `resp`
- Cambiare le ultime righe della funzione `predict` come segue:

```
@app.route("/predict")
def predict():
    ...
    output = model.predict([inputs])[0]
    response = "Insolvente" if output else "Non insolvente"
    return render_template("predict.html", resp=response)
```

- Ripetere le richieste con gli URL sopra, sarà visualizzata la pagina HTML definita al posto della semplice stringa

### Form per immissione dati

- Andiamo ora a creare un form per inserire i dati senza scrivere manualmente l'URL
- Creiamo un form che invii i dati all'URL della funzione `predict`
  - usiamo la funzione `url_for` di Flask per ottenere tale URL
- Inseriamo nel form un campo per ciascuna variabile
  - per `LIMIT_BAL` e `PAY_6` usiamo un campo di testo
  - per `EDUCATION`, essendo un attributo categorico, possiamo creare un menu di opzioni possibili che sono codificate nei rispettivi valori numerici
- Poniamo tale form in un template `index.html` da visualizzare come home page

In [17]:
%%writefile templates/index.html
<!DOCTYPE html>
<head>
 <title>Predizione default carte</title>
</head>
<body>
 <h2>Default carte di credito</h2>
 <form method="GET" action="{{ url_for("predict") }}">
  <p>Credito da saldare: <input name="LIMIT_BAL"></p>
  <p>Ritardo ultimo mese (-1=puntuale): <input name="PAY_6"></p>
  <p>Titolo di studio: <select name="EDUCATION">
   <option value="1">Laurea magistrale</option>
   <option value="2">Laurea breve</option>
   <option value="3">Diploma</option>
   <option value="4">Altro</option>
  </select></p>
  <p><button type="submit">Conferma</button></p>
 </form>
</body>
</html>

Overwriting templates/index.html


- Modifichiamo quindi la funzione `home` in modo che visualizzi tale template

```
@app.route("/")
def home():
    return render_template("index.html")
```

- Andare ora alla home page dell'applicazione: http://127.0.0.1:5000/
- Testare l'interfaccia inserendo ad es. i valori delle variabili che sono usati negli URL d'esempio sopra

## Esercizi suggeriti

- Considerare altre variabili per il modello e applicare preprocessing opportuno (es. one-hot encoding per `EDUCATION`)
- Modificare la pagina dei risultati in modo che visualizzi la probabilità di insolvenza piuttosto che una risposta sì/no
  - promemoria: usare il metodo `predict_proba` del modello di classificazione per ottenere le probabilità di ciascuna classe
- Verificare la correttezza dei dati in ingresso (es. `LIMIT_BAL >= 0`) e visualizzare un errore nella pagina in caso non siano corretti
- Creare una funzione che restituisca la risposta del modello in formato JSON piuttosto che come pagina Web e (per chi è pratico con JavaScript) far sì che alla conferma del form tale risposta sia visualizzata direttamente sotto il form senza ricaricare la pagina intera